<a href="https://colab.research.google.com/github/mgfernan/gsl_uab/blob/main/notebooks/mapmatching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Map-matching technique using online services

This practice is a hands-on activity that allows the usage of online services in the cloud to process GNSS raw measurement data and perform a map-matching.

## Jason

[Jason](https://jason.rokubun.cat)

[Jason Python bindings](https://pypi.org/project/jason-gnss/)

## Mapbox map-matching API

[Mapbox map-matching API](https://docs.mapbox.com/help/glossary/map-matching-api)




---


The first step we need to perform is to install several dependencies and import some Python packages 

In [ ]:
!pip install jason-gnss

In [23]:
import csv
import os
import requests
import zipfile

from jason_gnss import jason

os.environ['JASON_API_KEY'] = '<api_key_from_jason_user_area>'
os.environ['JASON_SECRET_TOKEN'] = '<token_from_jason_user_area>'


Download sample datafile containing GNSS raw measurements

In [14]:
gnss_data_sample='https://raw.githubusercontent.com/mgfernan/gsl_uab/main/map_matching/gnss_raw_measurements_smartphone_motorbike.rnx'
r = requests.get(gnss_data_sample)

rover_file = 'gnss_data_sample.rnx'
with open(rover_file, 'w') as fh:
  fh.write(r.content.decode('utf-8'))

In [15]:
result = jason.submit_process(rover_file)

In [ ]:
process_id = result[0]['id']
jason.get_status(process_id)

In [19]:
# Wait for a while for the process to finish, then download the results
result_file = jason.download_results(process_id)

In [20]:
with zipfile.ZipFile(result_file, 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
csv_file = f'SPP/{rover_file}.rnx_position_SPP.csv'

with open(csv_file, 'r') as fh:
  rows = csv.DictReader(fh)

  track = [(row['latitude(deg)'], row['longitude(deg)']) for row in rows]

track